# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
import pandas as pd
import time

In [25]:
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["Ironhack"]
c = db.get_collection("companies")
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [34]:
# Your Code
list(c.find({"name":"Babelgum"}, {"name":1, "_id":0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [38]:
# Your Code
query = {"number_of_employees":{"$gt": 5000}}
projection = {"name":1, "_id":0}
list(c.find(query, projection).limit(5))

[{'name': 'Facebook'},
 {'name': 'eBay'},
 {'name': 'Cisco'},
 {'name': 'Yahoo!'},
 {'name': 'Google'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [51]:
# Your Code
condition_1 = {"founded_year":{"$gte": 2000}}
condition_2 = {"founded_year":{"$lte": 2005}}
query = {"$and":[condition_1, condition_2]}
projection = {"name":1, "founded_year":1, "_id":0}
list(c.find(query, projection))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'foun

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [62]:
# Your Code
condition_1 = {"founded_year":{"$lt": 2000}}
condition_2 = {"total_money_raised":{"$gt": "$100M"}}
query = {"$and":[condition_1, condition_2]}
projection = {"name":1, "founded_year":1, "total_money_raised":1, "_id":0}
list(c.find(query, projection))

[{'name': 'eBay', 'founded_year': 1995, 'total_money_raised': '$6.7M'},
 {'name': 'Cisco', 'founded_year': 1984, 'total_money_raised': '$2.5M'},
 {'name': 'Yahoo!', 'founded_year': 1994, 'total_money_raised': '$6.8M'},
 {'name': 'Google', 'founded_year': 1998, 'total_money_raised': '$555M'},
 {'name': 'SideStep', 'founded_year': 1999, 'total_money_raised': '$32M'},
 {'name': 'Zazzle', 'founded_year': 1999, 'total_money_raised': '$46M'},
 {'name': 'Scripps Networks Interactive',
  'founded_year': 1994,
  'total_money_raised': '$10M'},
 {'name': 'Piczo', 'founded_year': 1990, 'total_money_raised': '$11M'},
 {'name': 'PayScale', 'founded_year': 1999, 'total_money_raised': '$31.1M'},
 {'name': 'Amazon', 'founded_year': 1994, 'total_money_raised': '$8M'},
 {'name': 'PayPal', 'founded_year': 1998, 'total_money_raised': '$197M'},
 {'name': 'AOL', 'founded_year': 1985, 'total_money_raised': '$1B'},
 {'name': 'Snapfish', 'founded_year': 1999, 'total_money_raised': '$43.5M'},
 {'name': 'eXpresso

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [81]:
# Your Code
condition_1 = {"founded_year":{"$lt": 2005}}
condition_2 = {"number_of_employees":{"$lt": 1000}}
query = {"$and":[condition_1, condition_2]}
projection = {"name":1,"number_of_employees":1, "founded_year":1, "_id":0}
list(c.find(query, projection).sort("number_of_employees", 1).limit(10))

[{'name': 'Fox Interactive Media',
  'number_of_employees': 0,
  'founded_year': 1979},
 {'name': 'Ticketmaster', 'number_of_employees': 0, 'founded_year': 1976},
 {'name': 'Eurekster', 'number_of_employees': 0, 'founded_year': 2004},
 {'name': 'MindTouch', 'number_of_employees': 0, 'founded_year': 2004},
 {'name': 'Skype', 'number_of_employees': 0, 'founded_year': 2003},
 {'name': 'Monster', 'number_of_employees': 0, 'founded_year': 1994},
 {'name': 'EditGrid', 'number_of_employees': 0, 'founded_year': 2003},
 {'name': 'Simpy', 'number_of_employees': 0, 'founded_year': 2004},
 {'name': 'Compete', 'number_of_employees': 0, 'founded_year': 2000},
 {'name': 'stylediary', 'number_of_employees': 0, 'founded_year': 2004}]

### 6. All the companies that don't include the `partners` field.

In [126]:
# Your Code
list(c.find({"partners": {"$nin": ["partners"]}}, {"name": 1, "partners": 1, "_id":0}))

[{'name': 'Wetpaint', 'partners': []},
 {'name': 'AdventNet', 'partners': []},
 {'name': 'Zoho', 'partners': []},
 {'name': 'Digg', 'partners': []},
 {'name': 'Facebook', 'partners': []},
 {'name': 'Omnidrive', 'partners': []},
 {'name': 'Postini', 'partners': []},
 {'name': 'Geni', 'partners': []},
 {'name': 'Flektor', 'partners': []},
 {'name': 'Fox Interactive Media', 'partners': []},
 {'name': 'Twitter', 'partners': []},
 {'name': 'StumbleUpon', 'partners': []},
 {'name': 'Gizmoz', 'partners': []},
 {'name': 'Scribd', 'partners': []},
 {'name': 'Slacker', 'partners': []},
 {'name': 'Lala', 'partners': []},
 {'name': 'Helio', 'partners': []},
 {'name': 'eBay', 'partners': []},
 {'name': 'MeetMoi', 'partners': []},
 {'name': 'Joost', 'partners': []},
 {'name': 'CBS', 'partners': []},
 {'name': 'Viacom', 'partners': []},
 {'name': 'Babelgum', 'partners': []},
 {'name': 'Plaxo', 'partners': []},
 {'name': 'Cisco', 'partners': []},
 {'name': 'Yahoo!', 'partners': []},
 {'name': 'Powerse

### 7. All the companies that have a null type of value on the `category_code` field.

In [127]:
# Your Code
query = {"category_code": {"$type": "null"}}
projection = {"name": 1, "category_code": 1, "_id":0}
list(c.find(query, projection))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Drigg', 'category_code': None},
 {'name': 'SpaceTime', 'category_code': None},
 {'name': 'Touch Clarity', 'category_code': None},
 {'name': 'MMDAYS', 'category_code': None},
 {'name': 'Inside Group', 'category_code': None},
 {'name': 'Repeater Store', 'category_code': None},
 {'name': 'Tapesh', 'category_code': None},
 {'name': 'iPersians', 'category_code': None},
 {'name': 'NewPersia', 'category_code': None},
 {'name': 'Pyra Labs', 'category_code': None},
 {'name': 'Feedmap', 'category_code': None},
 {'name': 'NuvoMedia', 'category_code': None},
 {'name': 'Intwine', 'category_code': None},
 {'name': 'The Weinstein Company', 'category_code': None},
 {'name': 'ExecuNet', 'category_code': None},
 {'name': 'A1Vacations', 'category_code': None},
 {'na

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [128]:
# Your Code
condition_1 = {"number_of_employees":{"$gte": 100}}
condition_2 = {"number_of_employees":{"$lt": 1000}}
query = {"$and":[condition_1, condition_2]}
projection = {"name":1, "number_of_employees":1, "_id":0}
list(c.find(query, projection))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [142]:
# Your Code
query = {"ipo.valuation_amount": {"$type": 1}}
projection = {"name": 1, "ipo": 1, "_id":0}
list(c.find(query, projection).sort("ipo.valuation_amount", -1))

[{'name': 'GREE',
  'ipo': {'valuation_amount': 108960000000.0,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Groupon',
  'ipo': {'valuation_amount': 12800000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NASDAQ:GRPN'}},
 {'name': 'Tencent',
  

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [160]:
# Your Code
list(c.find({"number_of_employees": {"$exists": 1}},
            {"name":1, "number_of_employees":1, "_id":0})
             .sort("number_of_employees", 1).limit(10))

[{'name': 'Omnidrive', 'number_of_employees': None},
 {'name': 'Slacker', 'number_of_employees': None},
 {'name': 'Helio', 'number_of_employees': None},
 {'name': 'Postini', 'number_of_employees': None},
 {'name': 'Flektor', 'number_of_employees': None},
 {'name': 'Viacom', 'number_of_employees': None},
 {'name': 'CBS', 'number_of_employees': None},
 {'name': 'StumbleUpon', 'number_of_employees': None},
 {'name': 'Gizmoz', 'number_of_employees': None},
 {'name': 'Lala', 'number_of_employees': None}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```